In [1]:
notes = {'C': 0, 'C#': 1, 'D': 2, 'Eb': 3, 'E': 4, 'F': 5, 'F#': 6, 'G': 7, 'G#': 8, 'A':9, 'Bb': 10, 'B': 11}
intervals = {'1': 0, 'b2': 1, '2': 2, '#2': 3, 'b3': 3, '3': 4, '#3': 5, 'b4': 4, '4': 5, '#4': 6, 'b5': 6, '5': 7, '#5': 8, 'b6': 8, '6': 9, '#6': 10, 'b7': 10, '7': 11, 'b9': 12, '9': 13, '#9': 14, 'b11': 14, '11': 15, '#11': 16, 'b13': 16, '13': 17, '#13': 18}
# scales = [{'name': 'Triad', 'harmony': ['1', '3', '5']}, {'name': 'Major', 'harmony': ['1', '2', '3', '4', '5', '6', '7']}, {'name': 'Minor', 'harmony': ['1', '2', 'b3', '4', '5', 'b6', 'b7']}]
standard_guitar_strings = ['e', 'B', 'G', 'D', 'A', 'E']

In [2]:
import csv
def load_shape_file(filename):
    shapes_file = {}
    with open(filename, newline='') as csv_file:
        shapes_reader = csv.reader(csv_file, delimiter=',', quotechar='#')
        for row in shapes_reader:
            harmony = []
            name = ""
            first = True
            for item in row:
                if first:  
                    name = item.strip()
                    first = False
                else:
                    harmony.append(item.strip())
            shapes_file[name] = harmony
    return shapes_file
    
f = load_shape_file('shapes.csv')
print(f.keys())

dict_keys(['TriadMaj', 'TriadMin', 'TriadAug', 'TriadDism', '7', '-7', 'Maj7', '-7b5', 'º7', 'Major', 'Minor', 'Pentatonic', 'PentatonicMaj', 'Diminished', 'Whole-half', 'Ionian', 'Dorian', 'Phrygian', 'Lydian', 'Mixolydian', 'Aeolian', 'Locrian'])


In [13]:
import copy
import csv 

class Finger:
    guitar_strings = ['e', 'B', 'G', 'D', 'A', 'E']
    strings_semitones = [24, 19, 15, 10, 5, 0]

    def __init__(self, semitone, function, string, freat, finger=""):
        self.semitone = semitone
        self.function = function
        self.string = string
        self.freat = freat
        self.finger = finger
        
    def dist(self, f):
        s_num = Finger.guitar_strings.index(self.string)
        f_num = Finger.guitar_strings.index(f.string)
        return (Finger.strings_semitones[s_num] + self.freat) - (Finger.strings_semitones[f_num] + f.freat)
        
    def __eq__(self, f):
        return self.freat == f.freat and self.string == f.string and self.finger == f.finger

    def __str__(self):
        return "{} ({}) --> [{}/{}]({})".format(self.semitone, self.function, self.string, self.freat, self.finger)

class Shape:
    def __init__(self, fingers):
        self.fingers = fingers
        self.impracticable = False

    def __add__(self, a):
        return Shape(self.fingers + a)
    
    def get_max_min_freat(self):
        min_f = 100
        max_f = -100
        for f in self.fingers:
            min_f = min(min_f, f.freat)
            max_f = max(max_f, f.freat)
        return max_f, min_f

    def set_fingering(self):
        self.__set_fingering__(False)
        if self.impracticable:
            self.__set_fingering__(True)
        
        return copy.deepcopy(self)

    def __set_fingering__(self, priority_4s):
        self.impracticable = False
        max_f, min_f = self.get_max_min_freat()
        last_finger = '0'
        
        if max_f - min_f <= 3:
            # one finger per freat
            for f in self.fingers:
                f.finger = str(f.freat - min_f + 1)

        elif (max_f - min_f == 4) and priority_4s:
            # 4th finger extension
            for f in self.fingers:
                f.finger = str(f.freat - min_f + 1)
                if f.finger == '5':
                    f.finger = "4s"
                    if last_finger == '4' or last_finger == '4s' or last_finger == '1s':
                        self.impracticable = True
                if f.finger == '1' and last_finger =='1s':
                    self.impracticable = True
                last_finger = f.finger
        
        elif (max_f - min_f == 4) and not priority_4s:
            # 4th finger extension
            for f in self.fingers:
                f.finger = str(f.freat - min_f)
                if f.finger == '0':
                    f.finger = "1s"
                    if last_finger == '1' or last_finger == '1s' or last_finger == '4s':
                        self.impracticable = True
                if f.finger == '1' and last_finger =='1s':
                    self.impracticable = True
                last_finger = f.finger
        
        elif max_f - min_f == 5:
            # 4th finger and 1st finger extensions
            for f in self.fingers:
                f.finger = str(f.freat - min_f)
                if f.finger == '5':
                    f.finger = "4s"
                    if last_finger == '4' or last_finger == '4s' or last_finger == '1s':
                        self.impracticable = True
                elif f.finger == '0':
                    f.finger = '1s'
                    if last_finger == '1' or last_finger == '1s' or last_finger == '4s':
                        self.impracticable = True
                if f.finger == '1' and last_finger =='1s':
                    self.impracticable = True
                last_finger = f.finger
        else:
            self.impracticable = True
    
    def get_extensions(self):
        assert len(self.fingers) > 0
        if self.fingers[0].finger == '':
            self.set_fingering()
        num_extensions = 0
        for f in self.fingers:
            if f.finger == '1s' or f.finger == '4s':
                num_extensions += 1
        return num_extensions

    def __lt__(self, shape):
        return self.get_max_min_freat()[1] < shape.get_max_min_freat()[1]

def create_shape(root, shape_type):
    shapes_file = load_shape_file('shapes.csv')
    harmony = shapes_file[shape_type]
    shape_notes = []
    for note in harmony:
        shape_notes.append((notes[root] + intervals[note]) % 12)
    return shape_notes, harmony

def semitone_to_freat(semitone, function, guitar_strings):
    fingers = []
    if not isinstance(semitone, list):
        semitone = [semitone]
    for n, h in zip(semitone, function):
        for string in guitar_strings:
            freat = n - notes[string.upper()]
            if freat < 0:
                freat += 12
            fingers.append(Finger(n, h, string, freat, None))
            if freat < 5:
                fingers.append(Finger(n, h, string, freat + 12, None))    
    return fingers

def scale_to_freatboard(root, shape_type):
    all_shapes = []
    scale, harmony = create_shape(root, shape_type)
    fingers = semitone_to_freat(scale, harmony, standard_guitar_strings)
    # roots = semitone_to_freat(scale[0], harmony[0], standard_guitar_strings[-3:])
        
    for r in fingers:
        if r.string == 'E':
            shape = Shape([r])
            fill_shape(shape, harmony, (harmony.index(r.function) + 1) % len(harmony), fingers, all_shapes, r.freat, r.freat)
        
    return all_shapes
        
def fill_shape(shape, harmony, pointer, fingers, all_shapes, min_freat, max_freat, depth=0):
    valid_fingers = []
    for f in fingers:
        if f.function == harmony[pointer] and f.dist(shape.fingers[-1]) > 0 and f.dist(shape.fingers[-1]) < 12 and abs(f.freat - shape.fingers[-1].freat) < 5:
            valid_fingers.append(f)
    if len(valid_fingers) > 0:
        for f in valid_fingers:
            new_min = min(min_freat, f.freat)
            new_max = max(max_freat, f.freat)
            if new_max - new_min < 5 and new_min > 0:
                fill_shape(shape + [f], harmony, (pointer + 1)%len(harmony), fingers, all_shapes, new_min, new_max, depth+1)
            elif shape.fingers[-1].string == 'e':
                all_shapes.append(shape)
    else:
        all_shapes.append(shape)

def filter_shapes(shapes):
    shapes.sort()
    for i, s in enumerate(shapes):
        extensions = s.get_extensions()
        if extensions >= 4:
            s.impracticable = True
        if not s.impracticable:
            # Remove same position with more extensions
            first_finger = s.fingers[0]
            for j, s2 in enumerate(shapes[i+1:]):
                if not s2.impracticable and s2.fingers[0] == first_finger:
                    if s2.get_extensions() >= extensions:
                        s2.impracticable = True
                    else:
                        s.impracticable = True

            # Remove same position with less notes in 6th string
            last_finger = s.fingers[-1]
            for j, s2 in enumerate(shapes[i+1:]):
                if not s2.impracticable and s2.fingers[-1] == last_finger and len(s2.fingers) != len(s.fingers):
                    same_shape = True
                    k = 1
                    while same_shape and k < min(len(s.fingers), len(s2.fingers)):
                        index = -1 - k
                        if s.fingers[index] == s2.fingers[index]:
                            k += 1
                        else:
                            same_shape = False
                    if same_shape:
                        if len(s2.fingers) < len(s.fingers):
                            s2.impracticable = True
                        else:
                            s.impracticable = True

In [8]:
from matplotlib import pyplot as plt

class DrawShape:
    STRINGS = ['e', 'B', 'G', 'D', 'A', 'E']
    TEXT=['none', 'function', 'finger']
    
    def __init__(self, min_freats=4, text='function'):
        self.min_freats = min_freats
        self.figure, self.axes = plt.subplots(dpi=80)
        self.freat_size = 20
        self.string_separation = 10
        self.text = text
        
    def __draw_freatboard__(self, freats, init_freat):
        if freats < self.min_freats:
            freats = self.min_freats
        for s in range(6): # strings
            self.axes.plot([0, freats*self.freat_size + self.freat_size], [s*self.string_separation, s*self.string_separation], '-', color='gray')
        for f in range(freats + 1): # freats
            self.axes.plot([self.freat_size*f, self.freat_size*f], [0, self.string_separation*5], 'k-')
        for e, t in enumerate(DrawShape.STRINGS): # string names
            self.axes.text(-self.string_separation + (self.string_separation/5), self.string_separation*5-(self.string_separation/10) - e*self.string_separation, t, fontsize=12)
        if init_freat <= 1: # if initial freat draw double line
            self.axes.plot([1, 1], [0, self.string_separation*5], 'k-')
        plt.axis('off')
        # Draw freat symbols
        freat_symbols = [3, 5, 7, 9, 12, 15, 17]
        for s in freat_symbols:
            x = (s - init_freat)*self.freat_size + self.freat_size/2
            if x > 0 and x < (freats + 1)*self.freat_size:
                if s == 12:
                    y = 1.5*self.string_separation
                    circle = plt.Circle((x, y), self.freat_size/10, color='lightgray', fill=True)
                    self.axes.add_artist(circle)
                    y = 3.5*self.string_separation
                    circle = plt.Circle((x, y), self.freat_size/10, color='lightgray', fill=True)
                    self.axes.add_artist(circle)
                else:
                    y = 2.5*self.string_separation
                    circle = plt.Circle((x, y), self.freat_size/10, color='lightgray', fill=True)
                    self.axes.add_artist(circle)

        # Add starting fret
        plt.text(8, -self.string_separation, str(init_freat), fontsize=12)
        self.axes.set_aspect(1)
        self.axes.set_xlim(-self.freat_size/4, (freats+1)*self.freat_size)
        self.axes.set_ylim(-self.freat_size, self.string_separation*7)
        
    def draw_shape(self, shape, init_freat=None):
        max_f, min_f = shape.get_max_min_freat()
        if init_freat is not None:
            min_f = init_freat    
        self.__draw_freatboard__(max_f - min_f, min_f)
        for f in shape.fingers:
            x = (f.freat - min_f)*self.freat_size + self.freat_size/2
            y = 5*self.string_separation - DrawShape.STRINGS.index(f.string)*self.string_separation
            if f.function == '1':
                circle = plt.Circle((x, y), self.freat_size/5, color='r', fill=True, zorder=2)
            else:
                circle = plt.Circle((x, y), self.freat_size/5, color='k', fill=True, zorder=2)
            self.axes.add_artist(circle)
            self.__add_finger_text__(shape, x, y, f)
        plt.show()

    def __add_finger_text__(self, shape, x, y, f):
        if self.text == DrawShape.TEXT[1]:
            if len(f.function) == 2:
                self.axes.text(x-2.75, y-1.5, f.function, fontsize=11, color='white')
            else:
                self.axes.text(x-1.75, y-1.5, f.function, fontsize=11, color='white')
        elif self.text == DrawShape.TEXT[2]:
            if len(f.finger) == 2:
                self.axes.text(x-2.75, y-1.5, f.finger, fontsize=11, color='white')
            else:
                self.axes.text(x-1.75, y-1.5, f.finger, fontsize=11, color='white')
                

In [16]:
import copy
from draw_freatboard import DrawFreatBoard

all_shapes = scale_to_freatboard('C', 'Major')
# all_shapes = scale_to_freatboard('G', 'Dorian')
print(len(all_shapes))
all_shapes.sort()
shapes_with_fingers = []
for i in range(len(all_shapes)):
    shapes_with_fingers.append(all_shapes[i].set_fingering())

filter_shapes(shapes_with_fingers)

for f in shapes_with_fingers:
    if not f.impracticable:
        DrawFreatBoard(min_freats=1, text=DrawShape.TEXT_FINGER)
       

ImportError: cannot import name 'DrawFreatBoard' from 'draw_freatboard' (/mnt/c/Users/narcis/Documents/Drive_My/Code/music/draw_freatboard.py)

In [ ]:
# Others

notes_inv = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B']
chords = [{'name': '', 'harmony': ['1', '3', '5']}, {'name': '-', 'harmony': ['1', 'b3', '5']}, {'name': 'Aug.', 'harmony': ['1', '3', '#5']}, {'name': 'º', 'harmony': ['1', 'b3', 'b5']}, {'name': 'Sus2', 'harmony': ['1', '2', '5']}, {'name': 'Sus4', 'harmony': ['1', '4', '5']}, {'name': '7', 'harmony': ['1', '3', '5', 'b7']}, {'name': 'Maj7', 'harmony': ['1', '3', '5', '7']}, {'name': '-7', 'harmony': ['1', 'b3', '5', 'b7']}, {'name': '-7b5', 'harmony': ['1', 'b3', 'b5', 'b7']}]


def create_chord(root, chord_type):
    chord_name = [i['name'] for i in chords]
    chord = chords[chord_name.index(chord_type)]
    chord_notes = []
    for note in chord['harmony']:
        chord_notes.append((notes[root] + intervals[note]) % 12)
    return chord_notes

def inv_notes(notes):
    inverted = []
    for note in notes:
        inverted.append(notes_inv[note])
    return inverted

def note_to_freat(note, guitar_strings):
    freats = []
    for n in note:
        for string in guitar_strings:
            freat = notes[n] - notes[string]
            if freat < 0:
                freat += 12
            freats.append((string, freat))
            if freat < 6:
                freats.append((string, freat + 12))    
    return freats

def create_scale(root, scale_type):
    scale_name = [i['name'] for i in scales]
    scale = scales[scale_name.index(scale_type)]
    scale_notes = []
    for note in scale['harmony']:
        scale_notes.append((notes[root] + intervals[note]) % 12)
    return scale_notes, scale['harmony']
    
def chord_to_freatboard(root, chord_type):
    all_shapes = []
    chord = create_chord(root, chord_type)
    root = note_to_freat(inv_notes([chord[0]]), guitar_bass_strings)
    others = note_to_freat(inv_notes(chord), guitar_high_strings)
    for fr in root:
        chord_pose = [fr]
        for finger in others:
            if abs(finger[1] - fr[1]) <= 5:
                chord_pose.append(finger)

        # mesure average freat
        pose = 0
        for f in chord_pose:
            pose += f[1]
        pose = pose / len(chord_pose)

        # remove fingers further than 3 freats from mean
        chord_pose_filtered = []
        for f in chord_pose:
            if abs(f[1] - pose) <= 3:
                chord_pose_filtered.append(f)

        # mesure average freat
        pose = 0
        for f in chord_pose:
            pose += f[1]
        pose = pose / len(chord_pose)

        # remove fingers further than 3 freats from mean
        chord_pose_filtered = []
        for f in chord_pose:
            if abs(f[1] - pose) < 3:
                chord_pose_filtered.append(f)
        
        # Reorder
        sorted_shape = []
        for s in standard_guitar_strings:
            for f in chord_pose_filtered:
                if f[0] == s:
                    if f not in sorted_shape:
                        sorted_shape.append(f)

        all_shapes.append(sorted_shape)
        print("Root {} (avg. freat {}): {}".format(fr, round(pose), sorted_shape))
    return all_shapes

